In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from multiprocessing import Pool
import datetime

import scrapers

In [2]:
import itertools
import tqdm.notebook as tqdm

def p_imap_unordered_set(pool, func, iterable):
    iterable = list(iterable)
    temp = tqdm.tqdm(pool.imap_unordered(func, iterable), total=len(iterable))
    return set(itertools.chain.from_iterable(temp))


the_time = lambda: datetime.datetime.now().time().strftime('%H:%M:%S')

In [3]:
riders_done = set()
teams_done = set()
results = set()
teams_to_do = set()

print(the_time())

with Pool(50) as p:
    top_mens_riders = p_imap_unordered_set(
        p, scrapers.scrape_top, itertools.product(range(1960, 2021), range(1, 4), [True])
    )
    top_womens_riders = p_imap_unordered_set(
        p, scrapers.scrape_top, itertools.product(range(2018, 2021), range(1, 4), [False])
    )

top_riders = top_mens_riders | top_womens_riders

riders_to_do = top_riders.copy()
for i in range(50):
    if not riders_to_do:
        break
    print(f"{the_time()} - iter {i+1}, riders_to_do: {len(riders_to_do)}")
    with Pool(50) as p:
        fresh_results = p_imap_unordered_set(p, scrapers.scrape_rider, riders_to_do)

    results.update(fresh_results)
    riders_done.update(riders_to_do)

    teams_to_do = set(team for _, team in fresh_results) - teams_done

    print(f"{the_time()} - teams_to_do: {len(teams_to_do)}")

    with Pool(50) as p:
        riders_to_do = p_imap_unordered_set(p, scrapers.scrape_team, teams_to_do) - riders_done

    teams_done.update(teams_to_do)

print(the_time())

16:36:09


  0%|          | 0/183 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

16:36:27 - iter 1, riders_to_do: 4766


  0%|          | 0/4766 [00:00<?, ?it/s]

...

17:39:52 - iter 9, riders_to_do: 8


  0%|          | 0/8 [00:00<?, ?it/s]

17:39:54 - teams_to_do: 0


0it [00:00, ?it/s]

17:39:54


In [5]:
with open("top_riders.txt", "w") as f:
    f.write("\n".join(str(i) for i in sorted(top_riders)))
    
with open("top_mens_riders.txt", "w") as f:
    f.write("\n".join(str(i) for i in sorted(top_mens_riders)))
    
with open("top_womens_riders.txt", "w") as f:
    f.write("\n".join(str(i) for i in sorted(top_womens_riders)))

df = pd.DataFrame(iter(results), columns=["rider", "team"])
df.to_csv("scraped.csv", index=False)